## EDA (updates from Heather's version)
- 1 - The variable of hospital length of stay is deleted from this version to avoid data leakage.
- 2 - The lab data is added to the dataset and the accuracy and f1 score on the training dataset and testing dataset slightly improve.
- 3 - The overall accuracy without lab data is 63.7% and the overall accuracy with lab data is 64.3% on the training dataset. The overall accuracy without lab data is 63.6% and the overall accuracy with lab data is 64.8% on the test dataset.
- 4 - The f1 score without lab data is 0.583 and the f1 score with lab data is 0.591 on the training dataset. The f1 score without lab data is 0.580 and the f1 score with lab data is 0.594.

In [1]:
import numpy as np
import pandas as pd

In [2]:
df = pd.read_csv("processed_data1.csv", index_col=0)

In [3]:
lab = pd.read_csv("lab.csv", index_col=0)

In [4]:
df = df.merge(lab, how='left', on = 'icustay_id')

In [5]:
df.head()

,ROW_ID,SUBJECT_ID,HADM_ID,ADMITTIME,DISCHTIME,DEATHTIME,ADMISSION_TYPE,ADMISSION_LOCATION,DISCHARGE_LOCATION,INSURANCE,...,INR_min,INR_max,PT_min,PT_max,SODIUM_min,SODIUM_max,BUN_min,BUN_max,WBC_min,WBC_max
0,1,2,163353,2138-07-17 19:04:00,2138-07-21 15:48:00,NaN,NEWBORN,PHYS REFERRAL/NORMAL DELI,HOME,Private,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.1,22.0
1,2,3,145834,2101-10-20 19:08:00,2101-10-31 13:58:00,NaN,EMERGENCY,EMERGENCY ROOM ADMIT,SNF,Medicare,...,1.3,1.7,13.5,15.7,136.0,153.0,41.0,53.0,11.3,24.4
2,3,4,185777,2191-03-16 00:28:00,2191-03-23 18:41:00,NaN,EMERGENCY,EMERGENCY ROOM ADMIT,HOME WITH HOME IV PROVIDR,Private,...,1.1,1.1,12.8,12.8,141.0,141.0,10.0,10.0,9.7,9.7
3,4,5,178980,2103-02-02 04:31:00,2103-02-04 12:15:00,NaN,NEWBORN,PHYS REFERRAL/NORMAL DELI,HOME,Private,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,13.9,13.9
4,5,6,107064,2175-05-30 07:15:00,2175-06-15 16:00:00,NaN,ELECTIVE,PHYS REFERRAL/NORMAL DELI,HOME HEALTH CARE,Medicare,...,1.0,1.4,12.6,14.6,134.0,138.0,62.0,65.0,10.6,10.6


In [6]:
df = df.drop(columns=['ROW_ID'])

In [7]:
# time in emergency department, if not enter, then 0
df['EDREGTIME'] = df['EDREGTIME'].fillna(0)
df['EDOUTTIME'] = df['EDOUTTIME'].fillna(0)
df['EDstay'] = pd.to_datetime(df.EDOUTTIME) - pd.to_datetime(df.EDREGTIME)

In [8]:
df.columns

Index(['SUBJECT_ID', 'HADM_ID', 'ADMITTIME', 'DISCHTIME', 'DEATHTIME',
       'ADMISSION_TYPE', 'ADMISSION_LOCATION', 'DISCHARGE_LOCATION',
       'INSURANCE', 'LANGUAGE', 'RELIGION', 'MARITAL_STATUS', 'ETHNICITY',
       'EDREGTIME', 'EDOUTTIME', 'DIAGNOSIS', 'HOSPITAL_EXPIRE_FLAG',
       'HAS_CHARTEVENTS_DATA', 'HeartRate_Min', 'HeartRate_Max',
       'HeartRate_Mean', 'SysBP_Min', 'SysBP_Max', 'SysBP_Mean', 'DiasBP_Min',
       'DiasBP_Max', 'DiasBP_Mean', 'TempC_Max', 'RespRate_Max',
       'RespRate_Mean', 'HeartRate_Mean_1', 'HeartRate_Min_1', 'Glucose_Max',
       'Glucose_Min', 'Glucose_Mean', 'icustay_id', 'INTIME', 'OUTTIME',
       'ICU_LOS', 'Height', 'Weight', 'GENDER', 'DOB', 'DOD', 'Hosp_LOS',
       'age', 'subject_id', 'hadm_id', 'ANIONGAP_min', 'ANIONGAP_max',
       'ALBUMIN_min', 'ALBUMIN_max', 'BANDS_min', 'BANDS_max',
       'BICARBONATE_min', 'BICARBONATE_max', 'BILIRUBIN_min', 'BILIRUBIN_max',
       'CREATININE_min', 'CREATININE_max', 'CHLORIDE_min', 'CHLORIDE

In [9]:
df['Hosp_LOS'] = pd.to_timedelta(df.Hosp_LOS).dt.total_seconds()
df['EDstay'] = df.EDstay.dt.total_seconds()

In [10]:
df_new = df.drop(['SUBJECT_ID', 'HADM_ID', 'icustay_id', 
                  'ADMITTIME', 'DISCHTIME', 'DEATHTIME', 'EDREGTIME', 'EDOUTTIME', 
                  'HOSPITAL_EXPIRE_FLAG', 'HAS_CHARTEVENTS_DATA', 
                  'HeartRate_Min', 'HeartRate_Max', 'SysBP_Min', 'SysBP_Max', 
                  'DiasBP_Min', 'DiasBP_Max', 'RespRate_Max', 'HeartRate_Mean_1',
                  'HeartRate_Min_1', 'Glucose_Max', 'Glucose_Min', 'INTIME', 'OUTTIME', 
                  'DOB', 'DOD', 'LANGUAGE', 'MARITAL_STATUS', 'Height'], axis=1)

In [11]:
df_new.columns

Index(['ADMISSION_TYPE', 'ADMISSION_LOCATION', 'DISCHARGE_LOCATION',
       'INSURANCE', 'RELIGION', 'ETHNICITY', 'DIAGNOSIS', 'HeartRate_Mean',
       'SysBP_Mean', 'DiasBP_Mean', 'TempC_Max', 'RespRate_Mean',
       'Glucose_Mean', 'ICU_LOS', 'Weight', 'GENDER', 'Hosp_LOS', 'age',
       'subject_id', 'hadm_id', 'ANIONGAP_min', 'ANIONGAP_max', 'ALBUMIN_min',
       'ALBUMIN_max', 'BANDS_min', 'BANDS_max', 'BICARBONATE_min',
       'BICARBONATE_max', 'BILIRUBIN_min', 'BILIRUBIN_max', 'CREATININE_min',
       'CREATININE_max', 'CHLORIDE_min', 'CHLORIDE_max', 'GLUCOSE_min',
       'GLUCOSE_max', 'HEMATOCRIT_min', 'HEMATOCRIT_max', 'HEMOGLOBIN_min',
       'HEMOGLOBIN_max', 'LACTATE_min', 'LACTATE_max', 'PLATELET_min',
       'PLATELET_max', 'POTASSIUM_min', 'POTASSIUM_max', 'PTT_min', 'PTT_max',
       'INR_min', 'INR_max', 'PT_min', 'PT_max', 'SODIUM_min', 'SODIUM_max',
       'BUN_min', 'BUN_max', 'WBC_min', 'WBC_max', 'EDstay'],
      dtype='object')

In [12]:
df_new.to_csv("dat_raw.csv")

In [13]:
df_new2 = df_new.drop(columns=['Weight','subject_id','hadm_id'])

In [14]:
df_new2.DISCHARGE_LOCATION.value_counts()

HOME                         16090
HOME HEALTH CARE             10924
SNF                           5545
REHAB/DISTINCT PART HOSP      5042
DEAD/EXPIRED                  4393
SHORT TERM HOSPITAL           1441
LONG TERM CARE HOSPITAL       1385
DISC-TRAN CANCER/CHLDRN H      589
DISCH-TRAN TO PSYCH HOSP       382
HOSPICE-HOME                   266
LEFT AGAINST MEDICAL ADVI      214
HOSPICE-MEDICAL FACILITY       115
OTHER FACILITY                  49
HOME WITH HOME IV PROVIDR       42
ICF                             31
DISC-TRAN TO FEDERAL HC         11
SNF-MEDICAID ONLY CERTIF         1
Name: DISCHARGE_LOCATION, dtype: int64

In [15]:
# target variable encoding
result = [] 
for value in df_new2['DISCHARGE_LOCATION']: 
    if value == 'DEAD/EXPIRED': 
        result.append(4) 
    elif 'HOME' in value: 
        result.append(1)
    elif value.startswith('SNF'): 
        result.append(2)
    else: 
        result.append(3) 
        
df_new2['target'] = result

In [16]:
df_new2.target.value_counts()

1    27322
3     9259
2     5546
4     4393
Name: target, dtype: int64

In [17]:
df_new2 = pd.get_dummies(df_new2, columns=['ADMISSION_TYPE'])

In [18]:
df_new2 = pd.get_dummies(df_new2, columns=['ADMISSION_LOCATION']).drop(columns=['ADMISSION_LOCATION_** INFO NOT AVAILABLE **'])

In [19]:
df_new2 = pd.get_dummies(df_new2, columns=['INSURANCE'])

In [20]:
df_new2 = pd.concat([df_new2, pd.get_dummies(df_new2['RELIGION'])], axis=1).drop(
    ['RELIGION', 'NOT SPECIFIED', 'CATHOLIC', 'PROTESTANT QUAKER',
       'UNOBTAINABLE', 'OTHER', "JEHOVAH'S WITNESS",
       'GREEK ORTHODOX', 'EPISCOPALIAN', 'CHRISTIAN SCIENTIST',
       'METHODIST', 'UNITARIAN-UNIVERSALIST', 'HEBREW',
        'BAPTIST', 'ROMANIAN EAST. ORTH',
       'LUTHERAN'], axis=1)

In [21]:
result = [] 
for value in df_new2['ETHNICITY']: 
    if 'ASIAN' in value: 
        result.append('ASIAN') 
    elif 'WHITE' in value: 
        result.append('WHITE')
    elif 'BLACK' in value: 
        result.append('BLACK')
    else: 
        result.append('ETHNICITY_Others') 
        
df_new2['ETHNICITY'] = result
df_new2 = pd.concat([df_new2, pd.get_dummies(df_new2['ETHNICITY'])], axis=1).drop(
    'ETHNICITY', axis=1)

In [22]:
result = [] 
for value in df_new2['DIAGNOSIS']: 
    if value == 'PNEUMONIA': 
        result.append('PNEUMONIA')
    elif value == 'CORONARY ARTERY DISEASE': 
        result.append('CORONARY ARTERY DISEASE')
    elif value == 'SEPSIS': 
        result.append('SEPSIS')
    elif value == 'INTRACRANIAL HEMORRHAGE': 
        result.append('INTRACRANIAL HEMORRHAGE')
    elif value == 'CHEST PAIN': 
        result.append('CHEST PAIN')
    elif value == 'CORONARY ARTERY DISEASE\CORONARY ARTERY BYPASS GRAFT /SDA': 
        result.append('CORONARY ARTERY DISEASE\CORONARY ARTERY BYPASS GRAFT /SDA')
    elif value == 'GASTROINTESTINAL BLEED': 
        result.append('GASTROINTESTINAL BLEED')
    elif value == 'CONGESTIVE HEART FAILURE': 
        result.append('CONGESTIVE HEART FAILURE')
    elif value == 'ALTERED MENTAL STATUS': 
        result.append('ALTERED MENTAL STATUS')
    else: 
        result.append('others') 
        
df_new2['DIAGNOSIS'] = result
df_new2 = pd.concat([df_new2, pd.get_dummies(df_new2['DIAGNOSIS'])], axis=1).drop(
    ['DIAGNOSIS', 'others'], axis=1)

In [23]:
df_new2 = pd.get_dummies(df_new2, columns=['GENDER'])

In [24]:
df_new2['ICU_LOS'] = df_new2['ICU_LOS'].fillna(0)

In [25]:
df_new2.drop(columns='Hosp_LOS', inplace=True)

In [26]:
from sklearn.model_selection import train_test_split

In [27]:
df_new2.drop(columns=['DISCHARGE_LOCATION'], inplace=True)

In [28]:
y = df_new2.pop('target')
X_train, X_test, y_train, y_test = train_test_split(df_new2, y, test_size=0.2, random_state=123, stratify=y)

In [29]:
from sklearn.impute import SimpleImputer
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.impute import KNNImputer

In [30]:
imp = SimpleImputer(strategy='mean')
imp.fit(X_train)
X_train1 = imp.transform(X_train)
X_test1 = imp.transform(X_test)

In [37]:
pd.DataFrame(X_train1,columns=X_train.columns).to_csv("X_train.csv")

In [38]:
pd.DataFrame(X_test1,columns=X_test.columns).to_csv("X_test.csv")

In [34]:
y_train.to_csv("y_train.csv")
y_test.to_csv("y_test.csv")

In [30]:
from sklearn.multiclass import OneVsRestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix
from sklearn.dummy import DummyClassifier

In [31]:
dummy_clf = DummyClassifier(strategy="most_frequent")

dummy_clf.fit(X_train1, y_train)
train_pred_dummy = dummy_clf.predict(X_train1)
test_pred_dummy = dummy_clf.predict(X_test1)

In [32]:
clf = OneVsRestClassifier(LogisticRegression(
    penalty='l1', solver='liblinear', multi_class='ovr', random_state=0))

clf.fit(X_train1, y_train)
train_pred = clf.predict(X_train1)
test_pred = clf.predict(X_test1)

In [33]:
print('Imputation 1:')
print('Training')
print('Accuracy: Dummy {}, LogReg {}'.format(
    accuracy_score(y_train, train_pred_dummy), accuracy_score(y_train, train_pred)))
print('F1: Dummy {}, LogReg {}'.format(
    f1_score(y_train, train_pred_dummy, average='weighted'), 
    f1_score(y_train, train_pred, average='weighted')))
print('Testing')
print('Accuracy: Dummy {}, LogReg {}'.format(
    accuracy_score(y_test, test_pred_dummy), accuracy_score(y_test, test_pred)))
print('F1: Dummy {}, LogReg {}'.format(
    f1_score(y_test, test_pred_dummy, average='weighted'), 
    f1_score(y_test, test_pred, average='weighted')))

Imputation 1:
Training
Accuracy: Dummy 0.5873280309544282, LogReg 0.6432179707652622
F1: Dummy 0.4346350712869246, LogReg 0.5905429878961517
Testing
Accuracy: Dummy 0.587274290627687, LogReg 0.6482158211521926
F1: Dummy 0.4345702497277467, LogReg 0.5947873841570391


In [34]:
conf_train = confusion_matrix(y_train, train_pred)
conf_train.diagonal()/conf_train.sum(axis=1)

array([0.9298655 , 0.26098715, 0.14810315, 0.38645418])

In [35]:
conf_test = confusion_matrix(y_test, test_pred)
conf_test.diagonal()/conf_test.sum(axis=1)

array([0.93905564, 0.26420198, 0.15550756, 0.3629124 ])